## Test Run of Proposed Protocol

Given the security deficiencies of using Python's `random` module, here is an alternative way for $N$ users to generate a shared random $D$ value for each run of the SAFE Protocol.

Steps:
* each user uses the `secret` module (which uses a Crytographically-Secure PRNG) to generate a binary string.
* this binary string is mapped to a float (in the case of `0` or `nan`, some default float value is used as their secret share.)
* users do additive secret sharing of their float values to come up with a shared $D$ value.

**Note:** the binary string generated from the `secret` module needs to be long enough for security (i.e., more than 256 bits), but we can only convert strings of 32 bits to floats. So, we randomly **sample** 32 bits from each bit-string generated by the CPRNG before converting it to some float.

<!--Initially, I thought I'd be able to use additive secret sharing to generate a shared D value. But turns out that (1) the Pysyft .share() method's API is nowhere to be found on the internet, (2) moreover sources seem to indicate that the PySyft wrapper for PyTorch is deprecated? At least for the secure MPC side of things. (3) Even if I could use the .share() method it doesn't seem like I could pass in more than 5 values without doing some really ugly hard coding.

This seems like a more promising approach: https://www.geeksforgeeks.org/what-is-secure-multiparty-computation/#:~:text=Some%20techniques%20used%20in%20secure%20Multiparty%20computation%20are,one%20gate%20at%20a%20time.%20...%20More%20items

Each user's random float (via mapping a binary string to some float) is their secret. They split this secret and distribute the shares to all other users.

Then, the users jointly calculate the average of the secret values. This constitutes the rounds D value.

The main drawback of this approach, and secure MPC protocols in general, is the communication overhead. For N users, each user has to communicate with N-1 other users. So, we have O(N^2) growth in communication overhead.

So, this method may only be acceptable for smaller settings.

Another consideration: in order to perform multi-party additive sharing, each user needs to have a shared and secret randomness parameter. This would be a large prime.

It's not clear that Tsung's protocol allows for users to jointly generate some large prime... 

What if a single user generated and broadcast the prime to all others for the round? ... Haiz. 

  -->


### Example Of D-Generation

In [ ]:
# Helper Functions To Convert Integers To Floats

"""
* Key Resources: (code adapted from)
# https://stackoverflow.com/questions/8751653/how-to-convert-a-binary-string-into-a-float-value
# https://stackoverflow.com/questions/25099626/convert-scientific-notation-to-float
# https://stackoverflow.com/questions/30971079/how-to-convert-an-integer-to-a-list-of-bits
"""

import math
from codecs import decode
import struct
import secrets
from random import sample

NUMBER_OF_USERS = 100
BIT_SECURITY = 256
DEFAULT_FLOAT = 1569832.524 # NOTE- just choosing some value. Not sure if there's a better way to do it.

# STEP 1: GET SOME BINARY STRINGS

r = secrets.SystemRandom()
# r.getrandbits returns an int by default, so we convert it to binary.
# https://docs.python.org/3/library/secrets.html <= NOTE : docs recommend at least 256 bits of randomnness as a baseline for security.
binary_strings = [bin(r.getrandbits(BIT_SECURITY))[2:] for i in range(NUMBER_OF_USERS)]
# however, we need can only convert 32-bit strings to floats, we we randomly sample 32 bits from these strings to generate the floats.
binary_strings = ["".join(sample(bit_string, 32)) for bit_string in binary_strings] 
# NOTE : for greater randomness, if we wanted to use `secrets` to randomly sample the bit_string, we could use `secrets.choice()` 32 times
#       to get a random bit from the bit_strings. However, that would be significantly slower. 


# OUTPUT BIT_STRINGS
for i in range(3):
    print(f'User {i}\'s 32-bit binary string: {binary_strings[i]}')
print("\n")

# STEP 2: MAP BIT STRINGS TO FLOAT SHARES
def bin_to_float(byte_string: str) -> str:
    """ Convert binary string to a float. """
    float_bytes = int(byte_string, 2).to_bytes(4, 'big')  
    # see https://docs.python.org/3/library/struct.html for manipulation of params
    float_string =  format(struct.unpack('>f', float_bytes)[0], 'f')
    result = abs(float(float_string))
    if result == 0.0 or math.isnan(result):
        return DEFAULT_FLOAT
    else:
        return result 
    
# testing D generation
users_float_values = [bin_to_float(bit_string) for bit_string in binary_strings]

for i in range(10):
    print(f'User {i} Float Value: {users_float_values[i]}')


See [here](100-float-values.txt) for listing of a 100 Random Float Values.

### Additive Secret Sharing To Get The Round's D Value

At this point, each user has some float value that represents their share in an additive secret scheme.

#### Fixed Point Encoding

Additive secret sharing protocols are done with integer values, but each user has randomly generated floats. 

<!-- generate the shared randomnness parameter. one way around this: encode each floating point as a fixed precision number: as in https://blog.openmined.org/what-is-secure-multi-party-computation/ and then perform the protocol.  -->

<!-- alternative: save on the overhead: generate random integers, generate a shared randommness parameter, and then do additive secret sharing to get an integer which will then eventually be converted into a float... >